In [4]:
#CREATE USER INPUT TO INTERECT WITH MODEL

import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

#DATASET
true_data = pd.read_csv(r"C:\Users\nh013\Desktop\fake and real news dataset\True.csv")
fake_data = pd.read_csv(r"C:\Users\nh013\Desktop\fake and real news dataset\Fake.csv")
true_data['label'] = 1
fake_data['label'] = 0
combined_data = pd.concat([true_data, fake_data], ignore_index=True)
combined_data = combined_data.sample(frac=1, random_state=42).reset_index(drop=True)
combined_data['text'] = combined_data['text'].str.lower()
combined_data['text'] = combined_data['text'].apply(lambda x: re.sub(r'[^\w\s]', '', str(x)))
combined_data['text'] = combined_data['text'].apply(lambda x: word_tokenize(x))
stop_words = set(stopwords.words('english'))
combined_data['text'] = combined_data['text'].apply(lambda x: [word for word in x if word not in stop_words])
lemmatizer = WordNetLemmatizer()
combined_data['text'] = combined_data['text'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])
combined_data['text'] = combined_data['text'].apply(lambda x: ' '.join(x))

# TF-IDF vVECTORIZATION
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(combined_data['text'])
y = combined_data['label']

# TRAIN MODEL
nb_classifier = MultinomialNB()
nb_classifier.fit(X, y)

# BUILD USER INPUT AND PREDICT THE CLASS TRUE OR FAKE 

user_input = input("Enter the news text: ")
preprocessed_input = re.sub(r'[^\w\s]', '', user_input.lower())
tokenized_input = word_tokenize(preprocessed_input)
filtered_input = [word for word in tokenized_input if word not in stop_words]
lemmatized_input = [lemmatizer.lemmatize(word) for word in filtered_input]
processed_input = ' '.join(lemmatized_input)
X_user = vectorizer.transform([processed_input])
prediction = nb_classifier.predict(X_user)

if prediction[0] == 1:
    print("The news is classified as TRUE.")
else:
    print("The news is classified as FAKE.")


Enter the news text: WASHINGTON (Reuters) - The head of a conservative Republican faction in the U.S. Congress, who voted this month for a huge expansion of the national debt to pay for tax cuts, called himself a “fiscal conservative” on Sunday and urged budget restraint in 2018. In keeping with a sharp pivot under way among Republicans, U.S. Representative Mark Meadows, speaking on CBS’ “Face the Nation,” drew a hard line on federal spending, which lawmakers are bracing to do battle over in January. When they return from the holidays on Wednesday, lawmakers will begin trying to pass a federal budget in a fight likely to be linked to other issues, such as immigration policy, even as the November congressional election campaigns approach in which Republicans will seek to keep control of Congress. President Donald Trump and his Republicans want a big budget increase in military spending, while Democrats also want proportional increases for non-defense “discretionary
The news is classifie